<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import-Packages-and-Data" data-toc-modified-id="Import-Packages-and-Data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Import Packages and Data</a></span><ul class="toc-item"><li><span><a href="#Data-Preprocessing" data-toc-modified-id="Data-Preprocessing-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Data Preprocessing</a></span></li><li><span><a href="#Feature-Engineering" data-toc-modified-id="Feature-Engineering-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Feature Engineering</a></span></li></ul></li><li><span><a href="#Embeddings" data-toc-modified-id="Embeddings-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Embeddings</a></span><ul class="toc-item"><li><span><a href="#Get-Max-Length-of-Tokens-and-Integer-Encode-the-Documents" data-toc-modified-id="Get-Max-Length-of-Tokens-and-Integer-Encode-the-Documents-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Get Max Length of Tokens and Integer Encode the Documents</a></span></li><li><span><a href="#Pad-Documents-to-Max-Length" data-toc-modified-id="Pad-Documents-to-Max-Length-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Pad Documents to Max Length</a></span></li><li><span><a href="#Load-in-GloVe-Vectors" data-toc-modified-id="Load-in-GloVe-Vectors-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Load in GloVe Vectors</a></span></li><li><span><a href="#Load-in-Embeddings" data-toc-modified-id="Load-in-Embeddings-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Load in Embeddings</a></span></li></ul></li><li><span><a href="#Modeling" data-toc-modified-id="Modeling-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Modeling</a></span><ul class="toc-item"><li><span><a href="#Define-Models" data-toc-modified-id="Define-Models-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Define Models</a></span></li><li><span><a href="#LSTM-Model-1" data-toc-modified-id="LSTM-Model-1-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>LSTM Model 1</a></span><ul class="toc-item"><li><span><a href="#Model-Fitting-and-Evaluation" data-toc-modified-id="Model-Fitting-and-Evaluation-3.2.1"><span class="toc-item-num">3.2.1&nbsp;&nbsp;</span>Model Fitting and Evaluation</a></span></li></ul></li><li><span><a href="#LSTM-Model-2" data-toc-modified-id="LSTM-Model-2-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>LSTM Model 2</a></span><ul class="toc-item"><li><span><a href="#Model2-Fitting-and-Evaluation" data-toc-modified-id="Model2-Fitting-and-Evaluation-3.3.1"><span class="toc-item-num">3.3.1&nbsp;&nbsp;</span>Model2 Fitting and Evaluation</a></span></li></ul></li><li><span><a href="#RNN-Model" data-toc-modified-id="RNN-Model-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>RNN Model</a></span></li></ul></li><li><span><a href="#The-first-LSTM-model-has-0.90-accuracy,-the-second-LSTM-model-has-0.35-accuracy,-and-the-RNN-model-has-0.79-accuracy.-Based-on-the-performance-of-the-three-models,-we-decdided-the-first-LSTM-model-as-our-final-model.-Since-it-has-the-highest-accuracy" data-toc-modified-id="The-first-LSTM-model-has-0.90-accuracy,-the-second-LSTM-model-has-0.35-accuracy,-and-the-RNN-model-has-0.79-accuracy.-Based-on-the-performance-of-the-three-models,-we-decdided-the-first-LSTM-model-as-our-final-model.-Since-it-has-the-highest-accuracy-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>The first LSTM model has 0.90 accuracy, the second LSTM model has 0.35 accuracy, and the RNN model has 0.79 accuracy. Based on the performance of the three models, we decdided the first LSTM model as our final model. Since it has the highest accuracy</a></span></li></ul></div>

## Import Packages and Data

In [1]:
import pandas as pd
import warnings
import string
import re
import nltk
from nltk.stem import WordNetLemmatizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from imblearn.combine import SMOTETomek
from sklearn.model_selection import train_test_split
from numpy import asarray
from numpy import zeros
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from keras.layers.recurrent import SimpleRNN, LSTM
from keras.layers import Flatten, Masking
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk_stopwords = set(stopwords.words('english'))
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_excel('Behold+product+data+04262021.xlsx')
df.head(1)

,product_id,brand,brand_category,name,details,created_at,brand_canonical_url,description,brand_description,brand_name,product_active
0,01EX0PN4J9WRNZH5F93YEX6QAF,Two,Unknown,Khadi Stripe Shirt-our signature shirt,NaN,2021-01-27 01:17:19.305 UTC,https://two-nyc.myshopify.com/products/white-k...,Our signature khadi shirt\navailable in black ...,Our signature khadi shirt\n\navailable in blac...,Khadi Stripe Shirt-our signature shirt,True


In [3]:
df = df[['product_id','brand','brand_category','name','description','product_active','details']]


**We will leave out created_at and brand_canonical_url columns in this project. Since description contains the same text as in brand_description, and name contains the same text as in brand_name, we will only use description and name in this case. Also around 85% of the details column are missing values, and the text contained in this columns are highly specific to each product, so we will also leave this column on in the model building process.**


In [4]:
df.head()

,product_id,brand,brand_category,name,description,product_active,details
0,01EX0PN4J9WRNZH5F93YEX6QAF,Two,Unknown,Khadi Stripe Shirt-our signature shirt,Our signature khadi shirt\navailable in black ...,True,NaN
1,01F0C4SKZV6YXS3265JMC39NXW,Collina Strada,Unknown,RUFFLE MARKET DRESS LOOPY PINK SISTINE TOMATO,Mid-length dress with ruffles and adjustable s...,True,NaN
2,01EY4Y1BW8VZW51BWG5VZY82XW,Cariuma,Unknown,IBI Slip On Raw Red Knit Sneaker Women,IBI Slip On Raw Red Knit Sneaker Women,False,NaN
3,01EY50E27A0P5V6KCW01XPDB43,Cariuma,Unknown,IBI Slip On Black Knit Sneaker Women,IBI Slip On Black Knit Sneaker Women,False,NaN
4,01EY6DWHC2W5HPNEGXKEJ4A1CX,Cariuma,Unknown,CATIBA PRO Skate Black Suede and Canvas Contra...,NaN,False,NaN


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61355 entries, 0 to 61354
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   product_id      61355 non-null  object
 1   brand           61355 non-null  object
 2   brand_category  60896 non-null  object
 3   name            61354 non-null  object
 4   description     51238 non-null  object
 5   product_active  61355 non-null  bool  
 6   details         9200 non-null   object
dtypes: bool(1), object(6)
memory usage: 2.9+ MB


### Data Preprocessing

**In preprocessing, we first converted all the text to lower case, and we removed punctuations, new line characters, common stopwords in English, words that contain only one letter, and more than one consecutive spaces. We also fill up the missing values with 'Unknown' tokens. Lastly, we performed lemmatization to group the words that share simialr meaning.**

In [6]:
columns = ['name', 'description','details']

# change text to lower case
for col in columns:
    df[col] = df[col].str.lower()

In [7]:
df['description'] = df['description'].str.replace("\n", " ")

In [8]:
df['details']=df['details'].str.replace("\n", " ")

In [9]:
#replace null values with UNKNOWN_TOKEN
df['brand_category'] = df['brand_category'].fillna('Unknown')
df['name'] = df['name'].fillna('Unknown')
df['description'] = df['description'].fillna('Unknown')
df['details']=df['details'].fillna('Unknown')

In [10]:
# Create functions used in preprocessing
def remove_punctuations(text):
    for punctuation in '!%"#$&\'()*+,-./:;<=>?@[\\]^_`{|}~':
        text = text.replace(punctuation, '')
    return text
def remove_stopwords(text):
    words = text.split(" ")
    temp = []
    for word in words:
        if word in nltk_stopwords:
            continue
        temp.append(word)
    cleaned = " ".join(temp)
    return cleaned
def lemmatize(text):
    lemmatizer = WordNetLemmatizer()
    words = text.split(" ")
    temp = []
    for word in words:
        res = lemmatizer.lemmatize(word)
        temp.append(res)
    cleaned = " ".join(temp)
    return cleaned
def more_preprocess(text):
    result = re.sub(r'—', ' ', text)
    result = re.sub(r'•', ' ', result)
    result = re.sub(r'\s+', ' ', result)
    result = re.sub(r'\b[A-Z]?pattern[A-Z]?\b', '', result)
    return result

### Feature Engineering

For feature engineering part, we created `origin`, `material`, `attribute_value`, `category`, and `colors` features. 

`origin`: corresponding to the place of production of the item, we fetched this information from the description column.

`material`: corresponding to the product materials, for example polyester, cotton, silk, etc.. 

`attribute_value`: We merge `attribute_value` from the additional tags table with the product table to include information on the product's occasion and style. 

`category`: corresponding to the category of the products like shoes, bags, and scarves.

`color`: corresponding to the colors of each product.

In [11]:
# Get the information about the product origins
df['origin'] = df['description'].str.extract(r'made in\s(?P<origin>\w+\s\w+)')
df['origin'] = df['origin'].fillna('Unknown')

In [12]:
for i in range(len(df)):
    df['origin'][i] = re.sub(r'(the usa|los angeles|new york|nyc fits|our brooklyn|nyc also|united states|la fits|the u|nyc ways|long beach|la 100|nyc 97)','usa', df['origin'][i])

for i in range(len(df)):
    if len(df['origin'][i]) > 1:
        df['origin'][i] =  df['origin'][i].split()[0]

In [13]:
df.origin.value_counts()

Unknown      52510
usa           5248
italy         1144
peru           348
china          334
             ...  
vn               1
bali             1
hand             1
dark             1
signature        1
Name: origin, Length: 104, dtype: int64

In [14]:
# Fetch information about the product materials
df['material'] = df['description'].str.extract(r'%\s(?P<material>\w+\s)')
df['material'] = df['material'].fillna('Unknown')

In [15]:
df.material.value_counts()

Unknown         41912
cotton           5060
viscose          1183
polyester        1152
elastane          930
                ...  
cottonmade          1
silkstriped         1
wooldry             1
lace                1
silkhand            1
Name: material, Length: 321, dtype: int64

In [16]:
# We performed the preprocessings after we retrieved information from the data for feature engineering
df['description'] = df['description'].apply(remove_punctuations)
df['description'] = df['description'].apply(remove_stopwords)
df['description'] = df['description'].apply(lemmatize)

In [17]:
df['name'] = df['name'].apply(remove_punctuations)
df['name'] = df['name'].apply(remove_stopwords)
df['name'] = df['name'].apply(lemmatize)

In [18]:
df['description'] = df['description'].apply(more_preprocess)

In [19]:
df['details'] = df['details'].apply(remove_punctuations)
df['details'] = df['details'].apply(remove_stopwords)
df['details'] = df['details'].apply(lemmatize)

In [21]:
# Import the additional tags data:
add = pd.read_csv('usc_additional_tags USC.csv')

In [22]:
add.head()

,product_id,product_color_id,attribute_name,attribute_value
0,01E5ZXP5H0BTEZT9QD2HRZJ47A,01E5ZXP5JCREDC7WJVMWHK5Q40,materialclothing,linenblend
1,01E5ZXP5H0BTEZT9QD2HRZJ47A,01E5ZXP5JCREDC7WJVMWHK5Q40,materialclothing,cottonblend
2,01E5ZXP5H0BTEZT9QD2HRZJ47A,01E5ZXP5JCREDC7WJVMWHK5Q40,style,modern
3,01E5ZXP5H0BTEZT9QD2HRZJ47A,01E5ZXP5JCREDC7WJVMWHK5Q40,style,businesscasual
4,01E5ZXP5H0BTEZT9QD2HRZJ47A,01E5ZXP5JCREDC7WJVMWHK5Q40,style,classic


In [23]:
# Group the attribute_value rows together for the same product_id
grouping = add.groupby(['product_id'], as_index = False).agg({'attribute_value': ' '.join})

In [24]:
# Merge the grouped additional tags data with the product data
df = df.merge(grouping, on = 'product_id',how = 'left')

In [25]:
df['attribute_value'] = df['attribute_value'].fillna('Unknown')

In [26]:
df.head()

,product_id,brand,brand_category,name,description,product_active,details,origin,material,attribute_value
0,01EX0PN4J9WRNZH5F93YEX6QAF,Two,Unknown,khadi stripe shirtour signature shirt,signature khadi shirt available black white ea...,True,Unknown,Unknown,Unknown,Unknown
1,01F0C4SKZV6YXS3265JMC39NXW,Collina Strada,Unknown,ruffle market dress loopy pink sistine tomato,midlength dress ruffle adjustable strap bias c...,True,Unknown,usa,rose,Unknown
2,01EY4Y1BW8VZW51BWG5VZY82XW,Cariuma,Unknown,ibi slip raw red knit sneaker woman,ibi slip raw red knit sneaker woman,False,Unknown,Unknown,Unknown,Unknown
3,01EY50E27A0P5V6KCW01XPDB43,Cariuma,Unknown,ibi slip black knit sneaker woman,ibi slip black knit sneaker woman,False,Unknown,Unknown,Unknown,Unknown
4,01EY6DWHC2W5HPNEGXKEJ4A1CX,Cariuma,Unknown,catiba pro skate black suede canvas contrast t...,Unknown,False,Unknown,Unknown,Unknown,Unknown


In [27]:
# Create the category variable
df['category'] = 'Unknown'

In [28]:
for i in range(len(df)):
    if len(re.findall(r"\b(skirt|jeans|pants|dress|cargo|shorts|overalls|trousers|leggings)", df['description'][i], flags=re.IGNORECASE)) >= 1:
        df['category'][i] = 'Bottom'
    elif len(re.findall(r"\b(shoe|boot|flat|heel|sneaker|sandal|loafer|espadrille|oxford|moccasin|monkstrap)", df['description'][i], flags=re.IGNORECASE)) >= 1:
        df['category'][i] = 'Shoe'
    elif len(re.findall(r"\b(scarf|scarves)", df['description'][i], flags=re.IGNORECASE)) >= 1:
        df['category'][i] = 'Scarf'
    elif len(re.findall(r"\b(bag|clutch|pouch|satchel|duffle|tote|messenger bag|shoulder bag)", df['description'][i], flags=re.IGNORECASE)) >= 1:
        df['category'][i] = 'Handbag'
    elif len(re.findall(r"\b(one piece|one-piece|all-in-one|jumpsuit|playsuit|dress)", df['description'][i], flags=re.IGNORECASE)) >= 1:
        df['category'][i] = 'One Piece'
        

In [29]:
for i in range(len(df)):
    if len(re.findall(r"\b(skirt|jeans|pants|dress|cargo|shorts|overalls|trousers|leggings)", df['name'][i], flags=re.IGNORECASE)) >= 1:
        df['category'][i] = 'Bottom'
    elif len(re.findall(r"\b(shoe|boot|flat|heel|sneaker|sandal|loafer|espadrille|oxford|moccasin|monkstrap)", df['name'][i], flags=re.IGNORECASE)) >= 1:
        df['category'][i] = 'Shoe'
    elif len(re.findall(r"\b(scarf|scarves)", df['name'][i], flags=re.IGNORECASE)) >= 1:
        df['category'][i] = 'Scarf'
    elif len(re.findall(r"\b(bag|clutch|pouch|satchel|duffle|tote|messenger bag|shoulder bag)", df['name'][i], flags=re.IGNORECASE)) >= 1:
        df['category'][i] = 'Handbag'
    elif len(re.findall(r"\b(one piece|one-piece|all-in-one|jumpsuit|playsuit|dress)", df['name'][i], flags=re.IGNORECASE)) >= 1:
        df['category'][i] = 'One Piece'

In [30]:
for i in range(len(df)):
    if len(re.findall(r"\b(skirt|jeans|pants|dress|cargo|shorts|overalls|trousers|leggings)", df['details'][i], flags=re.IGNORECASE)) >= 1:
        df['category'][i] = 'Bottom'
    elif len(re.findall(r"\b(shoe|boot|flat|heel|sneaker|sandal|loafer|espadrille|oxford|moccasin|monkstrap)", df['details'][i], flags=re.IGNORECASE)) >= 1:
        df['category'][i] = 'Shoe'
    elif len(re.findall(r"\b(scarf|scarves)", df['details'][i], flags=re.IGNORECASE)) >= 1:
        df['category'][i] = 'Scarf'
    elif len(re.findall(r"\b(bag|clutch|pouch|satchel|duffle|tote|messenger bag|shoulder bag)", df['details'][i], flags=re.IGNORECASE)) >= 1:
        df['category'][i] = 'Handbag'
    elif len(re.findall(r"\b(one piece|one-piece|all-in-one|jumpsuit|playsuit|dress)", df['details'][i], flags=re.IGNORECASE)) >= 1:
        df['category'][i] = 'One Piece'

In [31]:
df['category'].value_counts()

Unknown      36696
Bottom       11951
Shoe          8343
Handbag       2878
One Piece      989
Scarf          498
Name: category, dtype: int64

In [32]:
# Create the colors variable
df['colors'] = ''

In [33]:
for i in range(len(df)):
    for col in set(re.findall(r"(beige|black|blue|brown|burgundy|gold|gray|navy|green|neutral|orange|pink|purple|silver|teal|white|yellow)", df['description'][i], flags=re.IGNORECASE)):
        df['colors'][i] = df['colors'][i] + ' ' + col
    if df['colors'][i] == '':
        df['colors'][i] = 'Unknown'

In [34]:
for i in range(len(df)):
    for col in set(re.findall(r"(beige|black|blue|brown|burgundy|gold|gray|navy|green|neutral|orange|pink|purple|silver|teal|white|yellow)", df['details'][i], flags=re.IGNORECASE)):
        df['colors'][i] = df['colors'][i] + ' ' + col
    if df['colors'][i] == '':
        df['colors'][i] = 'Unknown'

In [39]:
df.head()

,product_id,brand,brand_category,name,description,product_active,origin,material,attribute_value,category,colors
0,01EX0PN4J9WRNZH5F93YEX6QAF,Two,Unknown,khadi stripe shirtour signature shirt,signature khadi shirt available black white ea...,True,Unknown,Unknown,Unknown,Unknown,white black
1,01F0C4SKZV6YXS3265JMC39NXW,Collina Strada,Unknown,ruffle market dress loopy pink sistine tomato,midlength dress ruffle adjustable strap bias c...,True,usa,rose,Unknown,Bottom,Unknown
2,01EY4Y1BW8VZW51BWG5VZY82XW,Cariuma,Unknown,ibi slip raw red knit sneaker woman,ibi slip raw red knit sneaker woman,False,Unknown,Unknown,Unknown,Shoe,Unknown
3,01EY50E27A0P5V6KCW01XPDB43,Cariuma,Unknown,ibi slip black knit sneaker woman,ibi slip black knit sneaker woman,False,Unknown,Unknown,Unknown,Shoe,black
4,01EY6DWHC2W5HPNEGXKEJ4A1CX,Cariuma,Unknown,catiba pro skate black suede canvas contrast t...,Unknown,False,Unknown,Unknown,Unknown,Shoe,Unknown


In [104]:
df.to_csv('product_data.csv')

In [3]:
#df=pd.read_csv('product_data.csv')

## Embeddings

**In classification, we only took the top 30 more frequently appeared brands, and encoded the rest as 'Other'.** 

In [105]:
brands = pd.read_csv('behold_brands.csv')

In [106]:
top30 = df.groupby('brand').agg({'product_id':'count'}).sort_values(by = 'product_id', ascending = False).head(30).index

In [107]:
for i in range(len(df)):
    if df['brand'][i] in top30:
        continue
    else:
        df['brand'][i] = 'Other'

In [108]:
df['total'] = (df['brand_category'] + ' ' + df['name'] + ' ' + df['description'] +\
               ' ' + df['origin']+ ' ' + df['material']+ ' ' + df['category']+ ' ' + df['colors']+\
               ' ' + df['attribute_value']+ ' ' + df['details']).apply(str)
X = df['total'].values

In [109]:
df['description'][59796]

'fit model 59½ wearing size xsmall fabric content 75 polyester 25 elastanecategory lolux detail bollinger pant contructed buttery soft lolux fabric feature simple waistband flared leg highrise style perfect wear studio throw post practicestyle var00434'

### Get Max Length of Tokens and Integer Encode the Documents

In [110]:
def get_max_token_length_per_doc(docs):
    return max(list(map(lambda x: len(x.split()), docs)))

In [111]:
max_length = get_max_token_length_per_doc(X)

In [112]:
tokenizer = Tokenizer(num_words=5000, oov_token="UNKNOWN_TOKEN")
tokenizer.fit_on_texts(X)

In [113]:
def integer_encode_documents(docs, tokenizer):
    return tokenizer.texts_to_sequences(docs)

In [114]:
X = integer_encode_documents(X, tokenizer)

### Pad Documents to Max Length

In [115]:
X = pad_sequences(X, maxlen=max_length, padding='post')

In [116]:
df["brand"] = df["brand"].astype('category')
df["brand"] = df["brand"].cat.codes

In [117]:
encoder = LabelEncoder()
labels = to_categorical(encoder.fit_transform(df['brand']))

In [118]:
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size = 0.2, random_state = 1)

In [119]:
VOCAB_SIZE = int(len(tokenizer.word_index) * 1.1)

### Load in GloVe Vectors

In [120]:
# Load in GloVe Vectors

def load_glove_vectors():
    embeddings_index = {}
    with open('glove.6B.100d.txt',encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
    print('Loaded %s word vectors.' % len(embeddings_index))
    return embeddings_index


embeddings_index = load_glove_vectors()

Loaded 400000 word vectors.


### Load in Embeddings

In [121]:
# create a weight matrix for words in training docs
embedding_matrix = zeros((VOCAB_SIZE, 100))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: # check that it is an actual word that we have embeddings for
        embedding_matrix[i] = embedding_vector

## Modeling

### Define Models

In [ ]:
# Define the models

def make_classification_rnn_model(plot=False):
    model = Sequential()
    model.add(Embedding(VOCAB_SIZE, 100, weights=[embedding_matrix], input_length=max_length, trainable=False))
    model.add(Masking(mask_value=0.0)) # masking layer, masks any words that don't have an embedding as 0s.
    model.add(SimpleRNN(units=64, input_shape=(1, max_length)))
    model.add(Dense(16))
    model.add(Dense(labels.shape[1], activation='softmax'))
    
    # Compile the model
    model.compile(
    optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    # summarize the model
    model.summary()
    
    if plot:
        plot_model(model, to_file='model.png', show_shapes=True)
    return model

def make_classification_lstm_model(plot=False):
    model = Sequential()
    model.add(Embedding(VOCAB_SIZE, 100, weights=[embedding_matrix], input_length=max_length, trainable=False))
    model.add(Masking(mask_value=0.0)) # masking layer, masks any words that don't have an embedding as 0s.
    model.add(LSTM(units=32, input_shape=(1, max_length)))
    model.add(Dense(16))
    model.add(Dense(labels.shape[1], activation='softmax'))
    
    # Compile the model
    model.compile(
    optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    # summarize the model
    model.summary()
    
    if plot:
        plot_model(model, to_file='model.png', show_shapes=True)
    return model

def define_model():
    
    model = Sequential()
    model.add(Embedding(VOCAB_SIZE, 100, weights=[embedding_matrix], input_length=max_length, trainable=False))
    model.add(Masking(mask_value=0.0)) # masking layer, masks any words that don't have an embedding as 0s.
    model.add(LSTM(64, return_sequences=True, dropout=0.2, recurrent_dropout=0.15, activation = 'relu'))
    model.add(LSTM(64, return_sequences=False, dropout=0.2, recurrent_dropout=0.15, activation = 'relu'))
    #model.add(Flatten())
    model.add(Dense(16, activation = 'relu'))
    model.add(Dense(labels.shape[1], activation='sigmoid'))
    
    # Compile the model
    model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

### LSTM Model 1

In [132]:
# Compile the model

model_m = make_classification_lstm_model()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 428, 100)          4996000   
_________________________________________________________________
masking_7 (Masking)          (None, 428, 100)          0         
_________________________________________________________________
lstm_10 (LSTM)               (None, 32)                17024     
_________________________________________________________________
dense_12 (Dense)             (None, 16)                528       
_________________________________________________________________
dense_13 (Dense)             (None, 31)                527       
Total params: 5,014,079
Trainable params: 18,079
Non-trainable params: 4,996,000
_________________________________________________________________


#### Model Fitting and Evaluation

In [133]:
# Fit the model

history = model_m.fit(X_train, y_train,validation_split = 0.1, epochs=5, verbose=1)

Epoch 1/5
1381/1381 [==============================] - 317s 226ms/step - loss: 2.1210 - accuracy: 0.4590 - val_loss: 0.9418 - val_accuracy: 0.7661
Epoch 2/5
1381/1381 [==============================] - 304s 220ms/step - loss: 0.8320 - accuracy: 0.7846 - val_loss: 0.6411 - val_accuracy: 0.8383
Epoch 3/5
1381/1381 [==============================] - 299s 216ms/step - loss: 0.5697 - accuracy: 0.8514 - val_loss: 0.5148 - val_accuracy: 0.8670
Epoch 4/5
1381/1381 [==============================] - 299s 217ms/step - loss: 0.4612 - accuracy: 0.8798 - val_loss: 0.4357 - val_accuracy: 0.8861
Epoch 5/5
1381/1381 [==============================] - 292s 212ms/step - loss: 0.3659 - accuracy: 0.9018 - val_loss: 0.3864 - val_accuracy: 0.8963


In [134]:
loss, accuracy = model_m.evaluate(X_test, y_test, verbose=1)
print('Accuracy: %f' % (accuracy*100))

384/384 [==============================] - 27s 70ms/step - loss: 0.3631 - accuracy: 0.9045
Accuracy: 90.449023


### LSTM Model 2

In [123]:

model = define_model()
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 428, 100)          4996000   
_________________________________________________________________
masking_4 (Masking)          (None, 428, 100)          0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 428, 64)           42240     
_________________________________________________________________
lstm_7 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dense_6 (Dense)              (None, 16)                1040      
_________________________________________________________________
dense_7 (Dense)              (None, 31)                527       
Total params: 5,072,831
Trainable params: 76,831
Non-trainable params: 4,996,000
_______________________________________

#### Model2 Fitting and Evaluation

In [124]:
history = model.fit(X_train, y_train,validation_split = 0.1, epochs=5, verbose=1)

Epoch 1/5
1381/1381 [==============================] - 1576s 1s/step - loss: 16326771841876.2754 - accuracy: 0.1790 - val_loss: 0.1786 - val_accuracy: 0.2622
Epoch 2/5
1381/1381 [==============================] - 1661s 1s/step - loss: 0.1446 - accuracy: 0.2782 - val_loss: 0.1261 - val_accuracy: 0.3082
Epoch 3/5
1381/1381 [==============================] - 1658s 1s/step - loss: 0.1226 - accuracy: 0.3406 - val_loss: 0.1212 - val_accuracy: 0.3270
Epoch 4/5
1381/1381 [==============================] - 1581s 1s/step - loss: 0.1185 - accuracy: 0.3503 - val_loss: 0.1180 - val_accuracy: 0.3559
Epoch 5/5
1381/1381 [==============================] - 1515s 1s/step - loss: 0.1158 - accuracy: 0.3557 - val_loss: 0.1168 - val_accuracy: 0.3663


In [125]:
loss, accuracy = model.evaluate(X_test, y_test, verbose=1)
print('Accuracy: %f' % (accuracy*100))

384/384 [==============================] - 68s 176ms/step - loss: 0.1198 - accuracy: 0.3547
Accuracy: 35.473880


### RNN Model

In [130]:
model_cl = make_classification_rnn_model()
history = model_cl.fit(X_train, y_train,validation_split = 0.1, epochs=5, verbose=1)
loss, accuracy = model_cl.evaluate(X_test, y_test, verbose=1)
print('Accuracy: %f' % (accuracy*100))


Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 428, 100)          4996000   
_________________________________________________________________
masking_6 (Masking)          (None, 428, 100)          0         
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 64)                10560     
_________________________________________________________________
dense_10 (Dense)             (None, 16)                1040      
_________________________________________________________________
dense_11 (Dense)             (None, 31)                527       
Total params: 5,008,127
Trainable params: 12,127
Non-trainable params: 4,996,000
_________________________________________________________________
Epoch 1/5
1381/1381 [==============================] - 276s 198ms/step - loss: 2.1373 - accuracy: 0.4484 

## The first LSTM model has 0.90 accuracy, the second LSTM model has 0.35 accuracy, and the RNN model has 0.79 accuracy. Based on the performance of the three models, we decdided the first LSTM model as our final model. Since it has the highest accuracy